# Bayesian Development
    v1 - n Prior; correct offset

In [12]:
# imports
from importlib import reload
import numpy as np

from astropy.table import Table
from astropy import units
from astropy.coordinates import SkyCoord

from frb.associate import bayesian

# Load sky and FRBs

In [29]:
sky = Table.read('tst_DES_180924.fits')
frbs = Table.read('tst_FRB_180924.fits')

In [30]:
frbs[0:5]

DES_r,ra,dec,iobj,obj_ra,obj_dec,theta
float64,float64,float64,int64,float64,float64,float64
19.58738499726639,325.2840199522759,-40.809129255163334,47578,325.284186,-40.809473,1.4229591462532893
19.883503965871313,325.4491267082537,-41.34527298969795,13026,325.448908,-41.345647,1.354869437609205
20.880999641715785,325.72853231402934,-41.653828544162295,19175,325.728522,-41.653498,1.6412871806252818
19.258669315368255,325.2453963834446,-40.6334153215558,54813,325.245823,-40.633118,1.5820113572493517
19.42883505943663,326.40971727957566,-41.701818635272296,211442,326.409368,-41.701616,1.2115197286530044


In [31]:
obj_coord = SkyCoord(ra=sky['ra'], dec=sky['dec'], unit='deg')
frb_coord = SkyCoord(ra=frbs['ra'], dec=frbs['dec'], unit='deg')

In [59]:
sky[0:5]

DES_g,DES_g_err,DES_r,DES_r_err,DES_i,DES_i_err,DES_z,DES_z_err,DES_Y,DES_Y_err,DES_ID,ra,dec,DES_tile,WISE_W1,WISE_W1_err,WISE_W2,WISE_W2_err,WISE_W3,WISE_W3_err,WISE_W4,WISE_W4_err
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,bytes12,float64,float64,float64,float64,float64,float64,float64,float64
22.4551,0.0337882,22.157,0.0371656,22.0062,0.0646035,21.3684,0.059213,22.0598,0.258294,212833823,324.8499,-41.2165,DES2137-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
23.194,0.0941275,21.9697,0.0437027,21.5887,0.0569069,21.1987,0.0581137,21.1634,0.150334,212835313,324.871578,-41.235802,DES2137-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
99.0,99.0,99.0,99.0,23.9364,0.253412,23.1386,0.178016,22.8145,0.353177,212835513,324.873778,-41.23811,DES2137-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
24.8797,0.26971,24.4296,0.255416,24.034,0.328238,23.7537,0.371262,24.2944,1.63749,212835530,324.87687,-41.238287,DES2137-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
26.0055,0.67688,24.9813,0.377289,24.5795,0.483049,24.5982,0.719402,24.5932,1.91446,212835629,324.875746,-41.239355,DES2137-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


# FRB 

## Uncertainty

In [82]:
sigR = 0.25 * units.arcsec

# Theta

In [107]:
theta_u = dict(method='uniform', max=4.) # This is twice what was actually generated

# n Prior

In [68]:
theta_max = 6 * units.arcsec

In [70]:
r_lim = 25.  # Assumed limiting magnitude

## One example

In [90]:
ifrb = 8

In [91]:
cands = frb_coord[ifrb].separation(obj_coord)  < theta_max
np.sum(cands)

5

In [92]:
cand_gal = sky[cands]
cand_gal

DES_g,DES_g_err,DES_r,DES_r_err,DES_i,DES_i_err,DES_z,DES_z_err,DES_Y,DES_Y_err,DES_ID,ra,dec,DES_tile,WISE_W1,WISE_W1_err,WISE_W2,WISE_W2_err,WISE_W3,WISE_W3_err,WISE_W4,WISE_W4_err
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,bytes12,float64,float64,float64,float64,float64,float64,float64,float64
25.8968,0.476234,24.586,0.225548,24.5381,0.341959,23.1215,0.275124,19.432,0.0160541,245078478,325.121359,-41.203248,DES2141-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
21.916,0.0649429,21.0832,0.0479341,20.9738,0.0687999,19.4988,0.0402104,16.3461,0.00495078,245078490,325.123018,-41.203783,DES2141-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
23.0426,0.119027,22.2848,0.0982046,22.336,0.158252,20.3258,0.0500422,17.3767,0.00856443,245078596,325.12318,-41.204758,DES2141-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
22.8341,0.0998012,22.0864,0.0837944,22.1234,0.130089,20.2562,0.0497385,17.4727,0.00940765,245078554,325.123534,-41.204249,DES2141-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
23.9085,0.193006,22.7417,0.110468,23.246,0.263104,21.0701,0.075993,19.3242,0.0366274,245078541,325.124585,-41.204037,DES2141-4123,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [93]:
frb_coord[ifrb].separation(obj_coord[cands]).to('arcsec')

<Angle [4.11963273, 1.13544809, 4.63313217, 3.29255709, 5.1022928 ] arcsec>

In [94]:
cand_coord = obj_coord[cands]
cand_coord

<SkyCoord (ICRS): (ra, dec) in deg
    [(325.121359, -41.203248), (325.123018, -41.203783),
     (325.12318 , -41.204758), (325.123534, -41.204249),
     (325.124585, -41.204037)]>

In [112]:
cand_sep = frb_coord[ifrb].separation(cand_coord).to('arcsec')
cand_sep

<Angle [4.11963273, 1.13544809, 4.63313217, 3.29255709, 5.1022928 ] arcsec>

### Raw $P(M_i)$

In [122]:
reload(bayesian)

<module 'frb.associate.bayesian' from '/home/xavier/Projects/FRB/frb/associate/bayesian.py'>

In [115]:
# Adding arbitrary half-light radius of 1.0"
cand_gal['r_half'] = 1.

In [123]:
raw_prior_Mi = bayesian.prior_Mi_n(cand_gal['DES_r'].data, cand_sep.to('arcsec').value, 
                                   cand_gal['r_half'].data, sigR.to('arcsec').value)
raw_prior_Mi

array([0.55107678, 0.98416379, 0.88772721, 0.93558569, 0.82316096])

### Raw $P(S)$

In [124]:
#raw_prior_S = bayesian.prior_S_n(r_lim, theta_max.to('arcsec').value)
raw_prior_S = 1. - np.max(raw_prior_Mi)
raw_prior_S

0.015836208639986582

### Renormalize priors to unit sum

In [125]:
prior_Mi, prior_S = bayesian.renorm_priors(raw_prior_Mi, raw_prior_S)
prior_Mi, prior_S

(array([0.13128532, 0.23446145, 0.21148696, 0.22288848, 0.19610507]),
 0.0037727260566120034)

### Calculate $p(x|M_i)$

In [128]:
reload(bayesian)

<module 'frb.associate.bayesian' from '/home/xavier/Projects/FRB/frb/associate/bayesian.py'>

In [129]:
p_xMi = bayesian.px_Mi(theta_max.to('arcsec').value, frb_coord[ifrb], cand_coord, theta_u, sigR.to('arcsec').value)
p_xMi

array([1.32521491e-03, 4.34308598e-03, 2.26092867e-05, 4.33184839e-03,
       1.98391998e-08])

### Calculate $p(x|S)$
    For our offset prior, the location doesn't matter provided it is within theta_max
    For ones that scale with theta, we might average over random placement..

In [133]:
p_xS = bayesian.px_Mi(theta_max.to('arcsec').value, frb_coord[ifrb], frb_coord[ifrb:ifrb+1], 
                      theta_u, sigR.to('arcsec').value)[0]
p_xS

0.004343085980399328

### Calculate $p(x)$

In [134]:
p_x = prior_S*p_xS + np.sum(prior_Mi * p_xMi)  #  I zeroed out P(S)!
p_x

0.0021789573237120507

### Evalute $P(M_i|x)$

In [135]:
P_Mix = prior_Mi * p_xMi / p_x
P_Mix

array([7.98461083e-02, 4.67327289e-01, 2.19442997e-03, 4.43110610e-01,
       1.78551805e-06])

In [1]:
P_Sx = prior_S * p_xS / p_x
P_Sx

NameError: name 'prior_S' is not defined

### Check the posteriors

In [138]:
np.sum(P_Mix) + p_Sx

1.0